In [2]:
import pandas as pd
import basicdescriptives_mod as bd
import re
import string
from unidecode import unidecode
from fuzzywuzzy import fuzz
import numpy as np
import locale
import os
import sys

In [104]:
current_dir = os.getcwd() # Obtener la ruta del directorio actual del notebook
ROOT_PATH = os.path.dirname(current_dir) # Obtener la ruta del directorio superior
sys.path.insert(1, ROOT_PATH) # Insertar la ruta en sys.path

import root # Importar el módulo root

df_path = root.DIR_DATA_RAW + 'db_raw_infousers.csv'

In [105]:
df = pd.read_csv(df_path)

## Manejo datos de fecha


In [106]:
df['FECHA DESEMBOLSO'] = df['FECHA DESEMBOLSO'].str.replace('/', '-')
df['FECHA DESEMBOLSO'] = pd.to_datetime(df['FECHA DESEMBOLSO'], errors='coerce', format='%m-%d-%Y')

In [107]:
df['PRÓXIMA FECHA PAGO'] = df['PRÓXIMA FECHA PAGO'].str.replace('/', '-')
df['PRÓXIMA FECHA PAGO'] = pd.to_datetime(df['PRÓXIMA FECHA PAGO'], errors='coerce', format='%m-%d-%Y')

## Procesamiento de texto variables categricas


In [108]:
df['CIUDAD RESIDENCIA'].nunique()

730

In [109]:
def processing_text(texto):
    if texto == np.nan:
        return np.nan
    texto = str(texto)
    ### Limpieza
    texto = texto.lower() # Estandarizar todo a minúscula
    texto = unidecode(texto)  # Eliminar tildes
    texto = re.sub(r'[^a-zA-Z0-9\s/-]', '', texto)
    texto = re.sub(r'\s+', ' ', texto) # Eliminar espacios en blanco adicionales
    texto = texto.strip() # Eliminar espacios al inicio y al final
    
    return texto  

In [110]:
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].map(processing_text)

## agrupacion de ciudades


In [111]:
df['CIUDAD RESIDENCIA'].nunique()

428

In [112]:
departamentos_colombia = [
    "amazonas",
    "antioquia",
    "arauca",
    "atlantico",
    "bolivar",
    "boyaca",
    "caldas",
    "caqueta",
    "casanare",
    "cauca",
    "cesar",
    "choco",
    "cordoba",
    "cundinamarca",
    "guainia",
    "guaviare",
    "huila",
    "la guajira",
    "magdalena",
    "meta",
    "narino",
    "norte de santander",
    "putumayo",
    "quindio",
    "risaralda",
    "san andres y providencia",
    "santander",
    "sucre",
    "tolima",
    "valle", "cauca",
    "vaupes",
    "vichada"
]


In [113]:
## procesamiento de texto extra para las ciudades
def processing_text_ciudades(texto):
    if pd.isna(texto):
        return np.nan
    texto = str(texto)
    ### Limpieza
    texto = texto.lower() # Estandarizar todo a minúscula
    texto = re.sub(r'\s+', ' ', texto) # Eliminar espacios en blanco adicionales
    texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto) # Eliminar caracteres especiales
    texto = texto.strip() # Eliminar espacios al inicio y al final
    
    # Tokenizar el texto
    palabras = texto.split()
    
    # Filtrar las palabras que no son nombres de departamentos
    palabras_filtradas = [palabra for palabra in palabras if not any(fuzz.ratio(palabra, depto) >= 80 for depto in departamentos_colombia)]
    
    # Reconstruir el texto
    texto_filtrado = ' '.join(palabras_filtradas)
    if not texto_filtrado:  # Si el texto filtrado está vacío
        return np.nan
    
    return texto_filtrado

In [114]:
df['CIUDAD RESIDENCIA'] = df['CIUDAD RESIDENCIA'].apply(processing_text_ciudades)

In [115]:
def mapear_ciudades(df):
    mapeo = {}

    for ciudad in df['CIUDAD RESIDENCIA']:
        if isinstance(ciudad, str):  # Verificar si es un string
            ciudad_estandar = None
            # Buscar si la ciudad ya está en el diccionario
            for ciudad_mapeada in mapeo:
                if fuzz.ratio(ciudad.lower(), ciudad_mapeada.lower()) > 70:  # Umbral de similitud del 80%
                    ciudad_estandar = ciudad_mapeada
                    break
            if ciudad_estandar:
                mapeo[ciudad_estandar].append(ciudad)
            else:
                mapeo[ciudad] = [ciudad]

    return mapeo

def obtener_ciudad_estandar(ciudad, mapeo):
    if pd.isna(ciudad):
        return np.nan
    if isinstance(ciudad, str):  # Verificar si es un string
        for ciudad_estandar, variantes in mapeo.items():
            if ciudad in variantes:
                return ciudad_estandar
    return ciudad

# Crear el mapeo
mapeo = mapear_ciudades(df)

# Crear una nueva columna en el DataFrame con la ciudad estandarizada
df['ciudad_estandarizada'] = df['CIUDAD RESIDENCIA'].apply(lambda x: obtener_ciudad_estandar(x, mapeo))


In [116]:
df['ciudad_estandarizada'].nunique()

258

In [117]:
cidades_path = root.DIR_DATA_RAW + 'ciudades.csv'
df_ciudades = pd.read_csv(cidades_path)

### Obtener longitud y latitud

In [118]:
def check_similarity_and_estandarize(x, cities_list):
    if not isinstance(x, str) or pd.isnull(x):
        return np.nan
    for city in cities_list:
        similarity = fuzz.ratio(x, city)
        if similarity >= 80:
            return city
    return np.nan

In [119]:
def check_similarity(x, cities_list):
    if not isinstance(x, str) or pd.isnull(x):
        return False
    return any(fuzz.ratio(x, city) >= 80 for city in cities_list)

In [120]:
def transform_string(x):
    cities = ['girardota','copacabana']
    for city in cities:
        if x == city:
            return 'bello'
    return x

In [121]:
df['ciudad_estandarizada'] = df['ciudad_estandarizada'].apply(transform_string)

In [122]:
df['En_lista'] = df['ciudad_estandarizada'].apply(check_similarity, cities_list=df_ciudades['name'].values)

In [123]:
df['En_lista'].dropna().value_counts()

En_lista
True     4048
False     125
Name: count, dtype: int64

In [124]:
df['ciudad_estandarizada'] = df['ciudad_estandarizada'].apply(check_similarity_and_estandarize, cities_list=df_ciudades['name'].values)

In [125]:
df['ciudad_estandarizada'].isnull().sum()

125

Imputar los que no estan en la lista con la ciudad que mas se repite

In [126]:
df['ciudad_estandarizada'].fillna(df['ciudad_estandarizada'].mode()[0], inplace=True)

C:\Users\jgome\AppData\Local\Temp\ipykernel_6636\2354175572.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ciudad_estandarizada'].fillna(df['ciudad_estandarizada'].mode()[0], inplace=True)


In [127]:
city_coords = df_ciudades.set_index('name')['location'].T.to_dict()

In [128]:
df['Ubicacion'] = df['ciudad_estandarizada'].map(city_coords)

In [129]:
regex = r'"latitude":(-?\d+\.\d+),"longitude":(-?\d+\.\d+)'
def extraer_latitud(ubicacion):
    if pd.isna(ubicacion):
        return None
    match = re.search(regex, ubicacion)
    if match:
        return float(match.group(1))
    else:
        return None

# Función para extraer longitud
def extraer_longitud(ubicacion):
    if pd.isna(ubicacion):
        return None
    match = re.search(regex, ubicacion)
    if match:
        return float(match.group(2))
    else:
        return None

# Crear columnas de latitud y longitud
df['latitud'] = df['Ubicacion'].apply(extraer_latitud)
df['longitud'] = df['Ubicacion'].apply(extraer_longitud)

In [130]:
df[['ciudad_estandarizada', 'latitud','longitud']].head(20)

,ciudad_estandarizada,latitud,longitud
0,pasto,1.21361,-77.28111
1,buga,3.90089,-76.29783
2,cucutilla,7.53941,-72.77238
3,bogota,4.60971,-74.08175
4,la celia,5.04333,-76.01667
5,cucutilla,7.53941,-72.77238
6,campoalegre,2.68489,-75.32311
7,medellin,6.25184,-75.56359
8,pereira,4.81333,-75.69611
9,bogota,4.60971,-74.08175


## Procesamiento tiempo en trabajo


In [131]:
def eliminar_valores_numericos_grandes(valor):
    if str(valor).isnumeric() and len(str(valor)) > 2:
        return np.nan
    return valor

In [132]:
df['TIEMPO TRABAJO'] = df['TIEMPO TRABAJO'].apply(eliminar_valores_numericos_grandes)

In [133]:
def agregar_anios(valor):
    if str(valor).isnumeric():
        return str(valor) + ' anos'
    return valor

df['TIEMPO TRABAJO'] = df['TIEMPO TRABAJO'].apply(agregar_anios)

In [134]:
# Separar datos con letras y números
df['TIEMPO TRABAJO'] = df['TIEMPO TRABAJO'].fillna('nan')
letras_numeros = df['TIEMPO TRABAJO'].str.contains(r'[a-zA-Z]', regex=True)
#letras_numeros = letras_numeros.fillna('nan')  # Rellenar NaN con False
df['letras_numeros'] = df['TIEMPO TRABAJO'][letras_numeros]
df['letras_numeros'] = df['letras_numeros'].replace('nan', np.nan)
df['fechas_trab'] = df['TIEMPO TRABAJO'][~letras_numeros]

In [135]:
def es_fecha(texto):
    if pd.isna(texto):
        return np.nan
    texto = re.sub(r'(\D)(\d)', r'\1 \2', texto)
    texto = re.sub(r'[^a-zA-Z0-9\s]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    
    
    meses = r'(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)'
    patron = rf'({meses} \d{{1,2}} de \d{{4}}|{meses} \d{{1,2}} \d{{4}}|{meses} – \d{{1,2}} – \d{{4}}|{meses}\d{{1,2}}-\d{{4}})'
    match = re.search(patron, texto)
    if match:
        return match.group()

df['fechas'] = df['letras_numeros'].apply(es_fecha)

In [136]:
df['fechas'].dropna().head()

264       marzo 1 de 2019
318          mayo 15 2018
2927       agosto 20 2000
3236    noviembre 19 2018
3769         julio 2 2015
Name: fechas, dtype: object

In [137]:
def obtener_valor_numerico(texto):
    if texto is not np.nan:
        texto = str(texto)
        texto = texto.lower()
        meses = 0
        anios = 0
        if 'mes' in texto:
            meses = re.findall(r'(\d+)\s*(?:mes?|meses?)', texto)
            if len(meses)==0:
                meses = 1
            else:    
                meses = int(meses[0])
                
        if any(word in texto.lower() for word in ['años', 'anos', 'año', 'años', 'ano']):
            anios = re.findall(r'(\d+)\s*(?:años?|anos?|año)', texto)
            if len(anios)==0:
                anios = 12
            else:    
                anios = 12*int(anios[0])
        if 'medio' in texto:
            meses = meses + 6
        return meses + anios
    return np.nan

In [138]:
df['letras_numeros'] = df['letras_numeros'].apply(obtener_valor_numerico)

In [139]:
df[['TIEMPO TRABAJO','letras_numeros']].head(5)

,TIEMPO TRABAJO,letras_numeros
0,24 meses,24.0
1,12 meses,12.0
2,1 ano,12.0
3,nan,NaN
4,9 horas,0.0


In [140]:
df[['TIEMPO TRABAJO','fechas_trab']].dropna().head(5)

,TIEMPO TRABAJO,fechas_trab
19,2018-01-15,2018-01-15
51,2021-03-01,2021-03-01
53,2020-02-04,2020-02-04
76,2019-05-27,2019-05-27
77,2019-07-23,2019-07-23


In [141]:
df['fechas_trab'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4173 entries, 0 to 4172
Series name: fechas_trab
Non-Null Count  Dtype 
--------------  ----- 
2544 non-null   object
dtypes: object(1)
memory usage: 32.7+ KB


In [142]:
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

df['fechas'] = df['fechas'].str.replace(' de ', ' ').str.capitalize()
df['fechas'] = pd.to_datetime(df['fechas'], format='%B %d %Y') 

In [143]:
df['fechas_trab'] = df['fechas_trab'].str.replace('/', '-')
df['TIEMPO TRABAJO FORMATO 1'] = pd.to_datetime(df['fechas_trab'], format='%Y-%m-%d', errors='coerce')
df['TIEMPO TRABAJO FORMATO 2'] = pd.to_datetime(df['fechas_trab'], format='%d-%m-%Y', errors='coerce')

In [144]:
df['TIEMPO TRABAJO FORMATO 2'] = df['TIEMPO TRABAJO FORMATO 2'].dt.strftime('%Y-%m-%d')

In [145]:
df['fechas_trab'] = df['TIEMPO TRABAJO FORMATO 1'].fillna(df['TIEMPO TRABAJO FORMATO 2'])

In [146]:
df['fechas_trab'] = df['fechas_trab'].fillna(df['fechas'])

In [147]:
df['meses_transcurridos'] = ((df['FECHA DESEMBOLSO'] - df['fechas_trab']) / pd.Timedelta(days=30.44))
df['meses_transcurridos'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4173 entries, 0 to 4172
Series name: meses_transcurridos
Non-Null Count  Dtype  
--------------  -----  
2497 non-null   float64
dtypes: float64(1)
memory usage: 32.7 KB


In [148]:
def convertir_a_nan(valor):
    if pd.isnull(valor) or valor < 0:
        return np.nan
    return valor

In [149]:
df['meses_transcurridos'] = df['meses_transcurridos'].apply(convertir_a_nan)
df['meses_transcurridos'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4173 entries, 0 to 4172
Series name: meses_transcurridos
Non-Null Count  Dtype  
--------------  -----  
2241 non-null   float64
dtypes: float64(1)
memory usage: 32.7 KB


In [150]:
df['meses_transcurridos'] = df['meses_transcurridos'].fillna(df['letras_numeros'])

In [151]:
df[['FECHA DESEMBOLSO','TIEMPO TRABAJO','meses_transcurridos']].head(20)



,FECHA DESEMBOLSO,TIEMPO TRABAJO,meses_transcurridos
0,2015-06-18,24 meses,24.0
1,2015-06-18,12 meses,12.0
2,2015-06-10,1 ano,12.0
3,2015-06-16,nan,NaN
4,2015-06-17,9 horas,0.0
5,2015-06-16,3 anos,36.0
6,2015-06-16,nan,NaN
7,2015-06-26,3 meses,3.0
8,2015-06-23,1 ano,12.0
9,2015-06-23,nan,NaN


In [152]:
df[['TIEMPO TRABAJO','meses_transcurridos']][df['meses_transcurridos']>1000]

,TIEMPO TRABAJO,meses_transcurridos
1742,154 anos,1848.0
1858,84 anos,1008.0
3170,84 anos,1008.0


In [155]:
df.columns

Index(['PLAZO', 'CAPITAL', 'INT CORRIENTE', 'FECHA DESEMBOLSO',
       'PRÓXIMA FECHA PAGO', 'DÍAS MORA', 'Cuotas en mora', 'TIPO EMPLEO',
       'CIUDAD RESIDENCIA', 'TRABAJO', 'TIPO DE VIVIENDA', 'ESTRATO',
       'AÑOS EN LA VIVIENDA', 'INGRESOS MENSUALES', 'GASTOS MENSUALES',
       'INGRESOS ADICIONALES', 'TIPO DE CONTRATO', 'PERIODO DE PAGO',
       'ESTADO CIVIL', 'NIVEL EDUCATIVO', 'PERSONAS A CARGO',
       'NUMERO DE HIJOS', 'TIPO DE VEHICULO', 'TIEMPO TRABAJO',
       'NUM.CREDITOS SOLICITADOS', 'USUARIO RECURRENTE',
       'ciudad_estandarizada', 'En_lista', 'Ubicacion', 'latitud', 'longitud',
       'letras_numeros', 'fechas_trab', 'fechas', 'TIEMPO TRABAJO FORMATO 1',
       'TIEMPO TRABAJO FORMATO 2', 'meses_transcurridos'],
      dtype='object')

In [156]:
## Columnas a eliminar
columas_to_drop = ['TIEMPO TRABAJO','fechas','fechas_trab','TIEMPO TRABAJO FORMATO 1',
                   'TIEMPO TRABAJO FORMATO 2','letras_numeros', 'FECHA DESEMBOLSO',
                   'CIUDAD RESIDENCIA','Ubicacion','TRABAJO','ciudad_estandarizada', 'En_lista',]
df.drop(columas_to_drop, axis=1, inplace=True)

# Guardado del df resultante

In [157]:
df.columns

Index(['PLAZO', 'CAPITAL', 'INT CORRIENTE', 'PRÓXIMA FECHA PAGO', 'DÍAS MORA',
       'Cuotas en mora', 'TIPO EMPLEO', 'TIPO DE VIVIENDA', 'ESTRATO',
       'AÑOS EN LA VIVIENDA', 'INGRESOS MENSUALES', 'GASTOS MENSUALES',
       'INGRESOS ADICIONALES', 'TIPO DE CONTRATO', 'PERIODO DE PAGO',
       'ESTADO CIVIL', 'NIVEL EDUCATIVO', 'PERSONAS A CARGO',
       'NUMERO DE HIJOS', 'TIPO DE VEHICULO', 'NUM.CREDITOS SOLICITADOS',
       'USUARIO RECURRENTE', 'latitud', 'longitud', 'meses_transcurridos'],
      dtype='object')

In [158]:
ruta1 = '../data/stage/db_pross_info_users.csv'
df.to_csv(ruta1, index=False) 